In [1]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import re
import pandas as pd

In [2]:
base_url = "https://www.thegreyhoundrecorder.com.au"
results_url = "https://www.thegreyhoundrecorder.com.au/results/"

In [3]:
def get_page_soup(url: str):
    r = Request(url, headers={"User-Agent": "Mozilla/5.0"})

    with urlopen(r) as webpage:
        content = webpage.read().decode()
        soup = BeautifulSoup(content)

    return soup

In [4]:
soup = get_page_soup("https://www.thegreyhoundrecorder.com.au/results/ballarat/82661")
results = soup.find("div", class_="resultsDesktopContent").find_all("div")

In [5]:
track_loc = soup.find("h1", class_="pageTitle").text.split()[0]
all_race_results = []
for r in results:
    result_entry = {}
    race_info = r.find("table", class_="raceHeader").find_all("td")
    race_no = int(race_info[0].text.split()[1])
    distance = int(race_info[2].text[:-1])
    grade = race_info[3].text
    fastest_split = race_info[5].text.split()[1].strip(',')

    try:
        fastest_split = float(fastest_split)
    except:
        fastest_split = None

    result_entry["track"] = track_loc
    result_entry["distance"] = distance
    result_entry["grade"] = grade
    result_entry["race_no"] = race_no
    result_entry["fastest_split"] = fastest_split

    race_result_info = r.find("table", class_="raceResultsTable") \
        .tbody.find_all("tr", recursive=False)
    
    runner_num = 0
    race_results = []
    for entry in race_result_info:
        data = entry.find_all("td", recursive=False)
        temp = {}
        try:
            temp["place"] = int(data[0].text)
        except:
            continue
        temp["place"] = int(data[0].text)
        temp["box"] = int(data[1].text)
        temp["dog_id"] = data[2].strong.a.text # lower case or uppercase?
        temp["time"] = float(data[4].text)
        temp["sect_time"] = float(data[6].text)
        temp["pir"] = re.sub("[^1-9]", '', data[7].text)
        temp["weight"] = float(data[8].text[:-2])
        race_results.append(temp)
        runner_num += 1
    
    result_entry["runner_num"] = runner_num
    result_entry["results"] = race_results
    all_race_results.append(result_entry)

In [6]:
all_race_results

[{'track': 'Ballarat',
  'distance': 390,
  'grade': 'Maiden',
  'race_no': 1,
  'fastest_split': 8.4,
  'runner_num': 8,
  'results': [{'place': 1,
    'box': 3,
    'dog_id': 'SWEET SENSATION',
    'time': 22.33,
    'sect_time': 8.4,
    'pir': '11',
    'weight': 28.4},
   {'place': 2,
    'box': 7,
    'dog_id': 'PYRENEES BLAZER',
    'time': 22.41,
    'sect_time': 8.61,
    'pir': '22',
    'weight': 28.8},
   {'place': 3,
    'box': 8,
    'dog_id': 'SMOOTH VIXEN',
    'time': 22.6,
    'sect_time': 8.73,
    'pir': '33',
    'weight': 26.1},
   {'place': 4,
    'box': 4,
    'dog_id': 'DEEP PURPLE',
    'time': 22.77,
    'sect_time': 8.76,
    'pir': '55',
    'weight': 27.5},
   {'place': 5,
    'box': 2,
    'dog_id': 'FLEUR BALE',
    'time': 22.78,
    'sect_time': 8.83,
    'pir': '64',
    'weight': 24.9},
   {'place': 6,
    'box': 6,
    'dog_id': 'TILLIGERRY TITAN',
    'time': 23.13,
    'sect_time': 8.95,
    'pir': '87',
    'weight': 31.1},
   {'place': 7,
    'b

In [7]:
a = pd.DataFrame(all_race_results)
a

,track,distance,grade,race_no,fastest_split,runner_num,results
0,Ballarat,390,Maiden,1,8.40,8,"[{'place': 1, 'box': 3, 'dog_id': 'SWEET SENSA..."
1,Ballarat,450,Mixed 6/7,2,6.59,8,"[{'place': 1, 'box': 6, 'dog_id': 'EDI BALE', ..."
2,Ballarat,390,Restricted Win,3,8.29,6,"[{'place': 1, 'box': 2, 'dog_id': 'SNAP SECOND..."
3,Ballarat,545,Grade 5,4,6.71,5,"[{'place': 1, 'box': 6, 'dog_id': 'APACHE SHOU..."
4,Ballarat,390,Grade 5,5,8.41,7,"[{'place': 1, 'box': 8, 'dog_id': 'UDELLAR BAL..."
5,Ballarat,390,Mixed 4/5,6,8.44,7,"[{'place': 1, 'box': 1, 'dog_id': 'OUR ELLY BY..."
6,Ballarat,450,Grade 5,7,6.61,8,"[{'place': 1, 'box': 4, 'dog_id': 'WHO TOLD ME..."
7,Ballarat,450,Mixed 4/5,8,6.61,7,"[{'place': 1, 'box': 8, 'dog_id': 'ORSON REIKO..."
8,Ballarat,450,S/E Heat,9,6.47,6,"[{'place': 1, 'box': 7, 'dog_id': 'MATT'S HOPE..."
9,Ballarat,450,S/E Heat,10,6.55,5,"[{'place': 1, 'box': 7, 'dog_id': 'BOLD CONTEN..."
